# CLASS PROPERLY FUNCTIONING:

This is the Molecule Class to analyze and give a template to analyze any single molecule computed with Gaussian. 
The files it requires to properly work are:
1. the output file of the computation called .out --> if you are used to say .log, change it on line 174
2. xyz of the structure you are interested in (If you want to easily get the xyz from the output file check the XYZ_converter.ipynb)
3. the formatted chk file (fchk file)

Furthermore this molecule class requires that the File_Name is the same for each type of file e.g (h2o.out, h2o.xyz, h2o.fchk) 


## CLASS METHODS


1. Bond_Length_Alternation_algoritm(BLA_list_of_Bonds) --> it gives the BLA angles you requested
BLA_list_of_Bonds --> [[1,2], [2,3]] n*2 Matrix with n the number of Bonds you want to put into the BLA ---> [Angstrom]

2. Dihedrals_Taker_V3(bond_list_dihedrals) --> it gives the Dihedral angles you requested
bond_list_dihedrals --> [[1,2,3,4], [5,6,3,4]] n*4 Matrix with n the number of Dihedrals you want [Degree]


3. Distances_Taker_V4(bond_list) --> it gives the Bonds length you requested
bond_list --> [[1,2], [2,3]] n*2 Matrix with n the number of Bonds you want [Angstrom]

4. Homo_Lumo_taker_better() --> it gives the HOMO and LUMO, electronegativity and Gap of the Molecule [eV]

5. molecules_energies() --> it gives the 4 possible thermochemistry energy of the output in order [Hartree]

6. polar_dipole_field_finder() --> it gives the polarizability, and dipole moment of the molecule [CHECK]

7. printer() --> it tells you which is the name of the file you putted as input

8. xyz_file_to_dataframe() --> it transform the xyz file into a Dataframe that it is used anywhere else [Angstrom]

In [48]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

class Molecule:
    
    def __init__(self, file_name):
        self.file_name = file_name
    
    def printer(self):
        return print(self.file_name)
    
    def xyz_Number_to_DF(self):
    
        atoms_gaus = list() #['blank'] # if i have gaussian xyz
        x = list() + [0]#['blank']
        y = list() + [0]#['blank']
        z = list() + [0]#['blank']

        # Read the xyz file
        with open(f'{self.file_name}.xyz') as fp:
            # Skip the first two lines.
            next(fp)
            next(fp)
            for line in fp:
                data = line.split()
                try:
                    atoms_gaus.append(int(data[0]))
                    x.append(float(data[1]))
                    y.append(float(data[2]))
                    z.append(float(data[3]))
                except:
                    break
    
        # create the DataFrame from xyz        
        atom_mapper = {1:"H", 2:"He", 
                   3:"Li", 4:"Be", 
                   5:"B", 6:"C", 7:"N", 8:"O", 9:"F", 10:"Ne",
                   11:"Na", 12:"Mg", 
                   13:"Al", 14:"Si", 15:"P", 16:"S", 17:"Cl", 18:"Ar", 
                   19:"K", 20:"Ca",
                   21:"Sc", 22:"Ti", 23:"V", 24:"Cr",25:"Mn",26:"Fe",27:"Co",28:"Ni",29:"Cu", 
                   30:"Zn",31:"Ga",32:"Ge",33:"As",34:"Se",35:"Br",36:"Kr",
                   37:"Rb",38:"Sr",39:"Y",40:"Zr",41:"Nb",42:"Mo",43:"Tc",44:"Ru",
                   45:"Rh",46:"Pd",47:"Ag",48:"Cd",49:"In",50:"Sn",51:"Sb",52:"Te",
                   53:"I",54:"Xe",55:"Cs",56:"Ba",57:"La",58:"Ce",59:"Pr",60:"Nd",
                   61:"Pm",62:"Sm",63:"Eu",64:"Gd",65:"Tb",66:"Dy",67:"Ho",68:"Er",
                   69:"Tm",70:"Yb",71:"Lu",72:"Hf",73:"Ta",74:"W",75:"Re",76:"Os",
                   77:"Ir",78:"Pt",79:"Au",80:"Hg",81:"Tl",82:"Pb",83:"Bi",84:"Po",
                   85:"At",86:"Rn",87:"Fr",88:"Ra",89:"Ac",90:"Th",91:"Pa",92:"U",
                   93:"Np",94:"Pu",95:"Am",96:"Cm",97:"Bk",98:"Cf",99:"Es",100:"Fm",
                   101:"Md",102:"No",103:"Lr",104:"Rf",105:"Db",106:"Sg",107:"Bh",
                   108:"Hs",109:"Mt",110:"Ds",111:"Rg",112:"Uub"
                  }     
    
        atoms = [] + [0]
        for i in atoms_gaus:
            atoms.append(atom_mapper[i])
    
        column = ['atom', 'x', 'y', 'z']
        frame = pd.DataFrame({column[0] : atoms,
                              column[1] : np.array(x),
                              column[2] : np.array(y),
                              column[3] : np.array(z)})
        return frame


    def xyz_Elements_to_DF(self): #xyz_to_dataframe_elements(self, file_name): 
        atoms = list() + [0]
        x = list() + [0]
        y = list() + [0]
        z = list() + [0]

        # Read the xyz file
    
        with open(f'{self.file_name}.xyz') as fp:
            # Skip the first two lines.
            next(fp)
            next(fp)
            for line in fp:
                data = line.split()
                try:
                    atoms.append(data[0])
                    x.append(float(data[1]))
                    y.append(float(data[2]))
                    z.append(float(data[3]))
                except:
                    break

        # create the DataFrame from xyz        
            
        column = ['atom', 'x', 'y', 'z']
        frame = pd.DataFrame({column[0] : atoms,
                          column[1] : np.array(x),
                          column[2] : np.array(y),
                          column[3] : np.array(z)})
    
        return frame
    
    
    def xyz_file_to_dataframe(self): 
        
        # Checking which xyz converter to use
        with open(f'{self.file_name}.xyz' ) as fp:
            # Skip the first two lines.
            next(fp), next(fp), next(fp)
            atom_type = fp.readline().split()[0]
            #atom_type = '5'
            try:
                int(atom_type)
                df = self.xyz_Number_to_DF()
            except:
                df = self.xyz_Elements_to_DF()
        
        return df
                

    def molecules_energies(self):
        with open(f'{self.file_name}.out') as fp:
            start = [line for line in fp]
            
        # here find what you want in the Output file:
        for i in range(len(start)):
            if ' Sum of electronic and zero-point Energies=' in start[i]:
                #data = start[i:i+6]
                Sum_elect_Zero_point_En =   float(start[i + 0].split()[-1])    # all in Hartree
                Sum_elect_thermal_En =      float(start[i + 1].split()[-1]) # all in Hartree
                Sum_elect_therm_Enthalpies= float(start[i + 2].split()[-1]) # all in Hartree
                Sum_elect_thermal_Free_En = float(start[i + 3].split()[-1]) # all in Hartree
            
        return Sum_elect_Zero_point_En, Sum_elect_thermal_En,Sum_elect_therm_Enthalpies, Sum_elect_thermal_Free_En
    
    
    def Distance_2_points(self, dataframe, position_1,position_2):    
    
        # Do the distance between 2 points in Cartesian coordinates
        atom1 = np.array(dataframe.loc[position_1][1:4]) #.loc[1][2]  # First [1] ROW, Second [2] Column
        atom2 = np.array(dataframe.loc[position_2][1:4])
        
        distance = math.dist(atom1, atom2)
    
        return distance
    
    
    def Bond_Length_Alternation_algoritm(self,BLA_kind):
        BLA_list_full = []
        BLA_list_neg = []
        count_neg = 0
        BLA_list_pos = []
        count_pos = 0
        dataframe = self.xyz_file_to_dataframe()
        # get the the distance between 2 points
        for i in range(len(BLA_kind)):
            position_1 = BLA_kind[i][0]
            position_2 = BLA_kind[i][1]
            # Calculate the distance between 2 points
            distance = self.Distance_2_points(dataframe, position_1,position_2)
            # Assign the right sign to any distances for the BLA
            element_bla = distance*((-1)**(i)) # START WITH POSITIVE --> SINGLE BOND
            # Divide positive and negative element of BLA 
            # Negative are DOUBLE BONDS (in this case but it doesn't matter ONLY the division Matter)
            if element_bla < 0:
                count_neg +=1
                BLA_list_neg.append(element_bla)
            # Positive are SINGLE BONDS
            if element_bla > 0:
                count_pos +=1
                BLA_list_pos.append(element_bla)
            # list that take all the bonds    
            BLA_list_full.append(element_bla)
            
        BLA = np.sum(np.array(BLA_list_neg))*(1/count_neg) + np.sum(np.array(BLA_list_pos))*(1/count_pos)
        # print(BLA_list_full) 
        return round(BLA,4)
    
    
    def Homo_Lumo_taker_better(self):
        starter = []
        with open(f'{self.file_name}.out') as fp:
            for i in fp:
                starter.append(i)
    
        # Python way to find the slice of text needed:
        #word_catch = '            Population analysis using the SCF density.\n'
        word_catch = ' The electronic state is 1-A.\n'
        key_index = starter.index(word_catch)
    
        # Take position (index) of the wanted line/words
        #all_indexes = []
        for i in range(len(starter)):
            given = starter[i]
            if given == word_catch:
                key_index = i
                #all_indexes.append(i)
    
        # anc copy it in this variable:
        part_needed = starter[key_index:key_index+60]

        i = 1
        while part_needed[i].split()[1] == 'occ.':
            i +=1
    
        #print('occupied:', part_needed[i-1], '\nvirtual:', part_needed[i]) 
        homo = float(part_needed[i-1].split()[-1])
        lumo = float(part_needed[i].split()[4])
        electroneg = round(((abs(homo) + abs(lumo))*0.5),5)
        delta_l_h = round((lumo - homo),5)
        #print('occupied:', homo, '\nvirtual:', lumo)
        
        return homo, lumo, electroneg, delta_l_h
    
    

    def Distances_Taker_V4(self, bond_list):
    
        # Create the xyz_frame inside:
        xyz_frame = self.xyz_file_to_dataframe()
    
        # Do the distance between 2 points in Cartesian coordinates
        # use code to calculate the distance between 2 points
        bond_length_list = [round(self.Distance_2_points(xyz_frame, bond_i[0], bond_i[1]), 5) for bond_i in bond_list]
        
        # here I am taking the elemnt i in the bond i The INPUT Must Be like: [[4,5]]
        #  elem_i_bond == 0 or 1
        atom_name = lambda bond_i, elem_i_bond: xyz_frame.loc[bond_i[elem_i_bond]][0] + str(bond_i[elem_i_bond])
        bond_atom_list = [f'{atom_name(bond_i, 0)}-{atom_name(bond_i, 1)}' for bond_i in bond_list]
        
        bonds_and_length = list( zip(bond_atom_list, bond_length_list))

    
        return bonds_and_length#bond_protagonist, round(bond_length,5)
    
    
    
    def dihedral_value(self, xyz_frame, position_0, position_1, position_2, position_3 ):
        # Create the xyz_frame inside:
        #xyz_frame = xyz_to_dataframe(file_name)
    
        # selected atoms:
        atom1 = np.array(xyz_frame.loc[position_0][1:4]) #.loc[1][2]  # First [1] ROW, Second [2] Column
        atom2 = np.array(xyz_frame.loc[position_1][1:4])
        atom3 = np.array(xyz_frame.loc[position_2][1:4])
        atom4 = np.array(xyz_frame.loc[position_3][1:4])
    
        # effective program:
        # Vectors Formation from points:
        v1 = atom2 - atom1
        v1 = list(v1)
        v2 = atom3 - atom2
        v2 = list(v2)
        v3 = atom4 - atom3
        v3 = list(v3)
        
        # Creation of the 2 Planes between i find the dihedral
        # plane 1:
        n1 = np.cross(v1, v2)
    
        #DEFINITION :plane1 = (n1[0]*(x-H4[0]) + n1[1]*(y-H4[1]) + n1[2]*(z-H4[2]))
        # plane 2:
        n2 = np.cross(v2, v3)
    
        # Dihedral Angle:
        dihedral_radian = np.arccos((n1[0]*n2[0] + n1[1]*n2[1] + n1[2]*n2[2])/
                                    (np.sqrt(n1[0]**2 + n1[1]**2 + n1[2]**2)*np.sqrt(n2[0]**2 + n2[1]**2 + n2[2]**2)))
    
        dihedral_degree = (dihedral_radian * 180)/np.pi
    
        return dihedral_degree


    def Dihedrals_Taker_V3(self, bond_list_dihedrals):
        # Create the xyz_frame inside:
        xyz_frame = self.xyz_file_to_dataframe()
    
        # Do the distance between 2 points in Cartesian coordinates
        dihedral_set = []
        for i in range(len(bond_list_dihedrals)):
            # 4 position of interest
            position_0 = bond_list_dihedrals[i][0]
            position_1 = bond_list_dihedrals[i][1]
            position_2 = bond_list_dihedrals[i][2]
            position_3 = bond_list_dihedrals[i][3]
        
            # use the program above
            dihedral_degree = self.dihedral_value(xyz_frame, position_0, position_1, position_2, position_3 )
    
            # This was just one cycle
            dihedral_set.append(dihedral_degree)

        # Write down which are the atoms between i did the angle
        dihedral_list = []
        for i in range(len(bond_list_dihedrals)):
            atom1_name = xyz_frame.loc[bond_list_dihedrals[i][0]][0] #.loc[1][2]  # First [1] ROW, Second [2] Column
            atom2_name = xyz_frame.loc[bond_list_dihedrals[i][1]][0]
            atom3_name = xyz_frame.loc[bond_list_dihedrals[i][2]][0]
            atom4_name = xyz_frame.loc[bond_list_dihedrals[i][3]][0]
        
            dihedral_name_part1 = atom1_name +str(bond_list_dihedrals[i][0]) +'-' + atom2_name + str(bond_list_dihedrals[i][1])
            dihedral_name_part2 = atom2_name+ str(bond_list_dihedrals[i][2])+ '-'+ atom4_name +  str(bond_list_dihedrals[i][3])
            dihedral_name = dihedral_name_part1 + '-' + dihedral_name_part2
        
            dihedral_list.append(dihedral_name)
        
        return dihedral_list, dihedral_set


    # ALL THIS IS FOR JUST ONE BRANCH
    def polar_dipole_field_finder(self):
        starter2 = []
        with open(f'{self.file_name}.fchk') as fp:
            for i in fp:
                starter2.append(i)
        #----------------------------------------
        #POLARIZABILITY SECTION
        #-----------------------------------------       
        # Find the index correlated to the key 
        #then you can employ it as a Searcher for that key 
        key = 'Polarizability                             R   N=           6\n'
        data = starter2.index(key)
        #take lines and copy them in a separated list
        polar = []
        for i in range(2):
            polar.append(starter2[data+i+1])
    
        # modify this list to obtain a valid matrix 
        polar_simm = []
        for i in range(2):
            for j in range(len(polar[i].split())):
                x = float(polar[i].split()[j])
                polar_simm.append(x)
        
        # create the polar matrix
        polar_matrix  = np.array([[polar_simm[0], polar_simm[1], polar_simm[3]],
                              [polar_simm[1], polar_simm[2], polar_simm[4]],
                              [polar_simm[3], polar_simm[4], polar_simm[5]]])

        # inverse of the polar matrix
        inverse_polar_matrix = np.linalg.inv(polar_matrix)
    
        # trace
        trace = (1/3)*np.sum([polar_simm[0],polar_simm[2],polar_simm[5]])

        #--------------------------------------------
        # DIPOLE MOMENT SECTION 
        #---------------------------------------------

        # Find the index correlated to the key 
        #then you can employ it as a Searcher for that key 
        key2 = 'Dipole Moment                              R   N=           3\n'
        data2 = starter2.index(key2)

        #take lines and copy them in a separated list
        dip = []
        dip.append(starter2[data2+1])
    
        # modify this list to obtain a valid vector 
        dipole_vector = []
    
        # modify this list to obtain a valid vector
        for i in range(1):
            for j in range(len(dip[i].split())):
                xx = float(dip[i].split()[j])
                dipole_vector.append(xx)
    
        # modulus
        dip_modulus = np.linalg.norm(dipole_vector)
    
        #-----------------------------------
        # INTERNAL ELECTRIC FIELD
        #----------------------------------

        internal_F = np.dot(inverse_polar_matrix,dipole_vector )
        # its modulus 
        F_modulus = np.linalg.norm(internal_F)
    
        #---------------------------------------------
        # ENERGY of a SINGLE BRANCH
        #--------------------------------------------
        U = -0.5 * np.dot(dipole_vector,internal_F )
    
        #print(dipole_vector)
        return dip_modulus,trace, F_modulus

## Molecule Class Examples:

In [49]:
molecule1 = Molecule('SIG779_close')
molecule1.printer()

SIG779_close


In [35]:
bond_list_dihedrals = [[1,2,3,4], [5,6,3,4]]
molecule1.Dihedrals_Taker_V3(bond_list_dihedrals)

(['C1-S2-S3-C4', 'C5-C6-C3-C4'], [10.186779396816108, 148.66138149878287])

In [51]:
bond_list = [[1,2], [5,3]]
molecule1.Distances_Taker_V4( bond_list)

[('C1-S2', 1.87206), ('C5-C3', 2.35414)]

In [52]:
molecule1.polar_dipole_field_finder()

(2.942070867814185, 546.8153033333333, 0.0031545928442059013)

In [53]:
molecule1.Homo_Lumo_taker_better()

(-0.19675, -0.11847, 0.15761, 0.07828)

In [54]:
BLA_kind = [[3,4],[4,5],[5,6],[6,7],[17,7],[21,17],[20,21]]
molecule1.Bond_Length_Alternation_algoritm(BLA_kind)

-0.0625

In [55]:
molecule1.molecules_energies()

(-3168.985893, -3168.949631, -3168.948687, -3169.057298)

In [56]:
molecule1.xyz_file_to_dataframe().head()

,atom,x,y,z
0,0,0.000000,0.000000,0.000000
1,C,0.216523,-0.472841,-0.294483
2,S,1.393892,-1.867319,0.122479
3,C,2.789818,-0.764448,0.083981
4,C,2.434921,0.557019,0.012451


### If you want then to deal with more molecules to compare them, check the next repository (once it is ready)